In [ ]:
'''
import re
import torch
report = !python3 -m deepspeed.env_report
r = re.compile('.*ninja.*OKAY.*')
assert any(r.match(line) for line in report) == True, "DeepSpeed Inference not correct installed"

# check cuda and torch version
torch_version, cuda_version = torch.__version__.split("+")
torch_version = ".".join(torch_version.split(".")[:2])
cuda_version = f"{cuda_version[2:4]}.{cuda_version[4:]}"
r = re.compile(f'.*torch.*{torch_version}.*')
assert any(r.match(line) for line in report) == True, "Wrong Torch version"
r = re.compile(f'.*cuda.*{cuda_version}.*')
assert any(r.match(line) for line in report) == True, "Wrong Cuda version"
'''


In [ ]:
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
)

In [ ]:

MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


In [ ]:
MODEL_TYPES

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("PygmalionAI/pygmalion-6b")

model = AutoModelForCausalLM.from_pretrained("PygmalionAI/pygmalion-6b")

In [ ]:
model.base_model.type

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    StoppingCriteriaList,
    MaxLengthCriteria,
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from datasets import load_dataset

In [ ]:
dataset = load_dataset("smangrul/MuDoConv")

In [ ]:
len(dataset["train"])

In [ ]:
dataset.keys()

In [ ]:
from transformers import TrainingArguments



In [ ]:
from transformers import AutoConfig

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


In [ ]:
config = AutoConfig.from_pretrained("/tmp/deepspeed_zero_stage2_accelerate_test/")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/tmp/deepspeed_zero_stage2_accelerate_test/")
tokenizer = AutoTokenizer.from_pretrained("/tmp/deepspeed_zero_stage2_accelerate_test/")

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

#model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("PygmalionAI/pygmalion-350m")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("PygmalionAI/pygmalion-350m")

In [ ]:
model.half()
model.to("cuda")

In [ ]:
# Print output from model

In [ ]:
model.generate

In [ ]:
tokenized_items

In [ ]:
prompt = "This is a test"
tokenized_items = tokenizer(prompt, return_tensors="pt").to("cuda")
        
logits = model.generate(
                     inputs=tokenized_items["input_ids"]
                    )
        
output = tokenizer.decode(logits[0], skip_special_tokens=True)
# Filter input
output = output[len(prompt)+1:]

In [ ]:
output

In [ ]:
prompt = "This is a test"
tokenized_items = tokenizer(prompt, return_tensors="pt").to("cuda")
        

logits_processor = LogitsProcessorList(
    [
        MinLengthLogitsProcessor(10, eos_token_id=model.generation_config.eos_token_id),
    ]
)
stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=20)])


In [ ]:
outputs = model.greedy_search(
    tokenized_items["input_ids"], logits_processor=logits_processor, stopping_criteria=stopping_criteria
)

In [ ]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
def get_response(model, tokenizer, prompt):
    prompt = "This is a test"
tokenized_items = tokenizer(prompt, return_tensors="pt").to("cuda")
        

logits_processor = LogitsProcessorList(
    [
        MinLengthLogitsProcessor(10, eos_token_id=model.generation_config.eos_token_id),
    ]
)
stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=20)])

                 

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(8))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(8))
small_train_dataset.shape

In [ ]:

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
#!pip install accelerate

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()